In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Vitamin_D_Levels/GSE129604'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Acute Effects of Single Doses of Bonito Fish Peptides and Vitamin D on Whole Blood Gene Expression Levels"
!Series_summary	"Fish contains high quality proteins and essential nutrients including vitamin D (VitD3). Fish peptide consumption can lower cardiovascular disease (CVD) risk factors and studies showed an association between VitD3 deficiency, CVD and CVD risk factors such as diabetes. This study investigated acute effects of a single dose of VitD3, bonito fish peptide hydrolysate (BPH), or a combination of both on CVD risk factors and whole blood gene expression levels. A randomized, crossover, placebo controlled trial was conducted in 22 adults. They ingested, in random order and at 7-day intervals, 1000 IU of VitD3, 3g of BPH, a combination of both, or a placebo. A 180 min oral glucose tolerance test was done. Differences in whole-genome expression levels after vs. before each supplementation were computed for 18 subjects. We observed that 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check if gene expression data is available
background_summary = '''
"Fish contains high quality proteins and essential nutrients including vitamin D (VitD3). Fish peptide consumption can lower cardiovascular disease (CVD) ...
'''
is_gene_available = True  # Since the study description confirms the measurement of gene expression data.

# Step 2: Map keys and define conversion functions

# Gender Mapping
gender_row = 0  # mapped to row representing 'Sex: Male' and 'Sex: Female'
def convert_gender(value):
    gender = value.split(':')[1].strip().lower()  # extracting and cleaning
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    return None

# Vitamin D Levels Mapping
trait_row = None  # No explicit mention of 'Vitamin_D_Levels' in the provided dictionary
def convert_trait(value):
    return None  # Since trait_row is None

# Age Mapping: Not found in dictionary, hence setting to None
age_row = None
def convert_age(value):
    return None  # Since age_row is None

# Save Metadata
save_cohort_info('GSE129604', './preprocessed/Vitamin_D_Levels/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, skip the clinical feature extraction step.
